In [191]:
import pandas as pd
import textwrap

In [192]:
# Read dataset.
def load_dataset(dataset_path: str) -> pd.DataFrame:
    df = pd.read_csv(dataset_path)
    df.dropna(inplace=True) # Drop null values (if any)
    return df

In [193]:
df_test = load_dataset(r'./data/cnn_dailymail/test.csv')
df_test['highlights'] = df_test['highlights'].str.replace('\n', ' ', regex=True)
df_test['highlights'] = df_test['highlights'].str.replace(' \.', '.', regex=True)
df_test.head()

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


In [194]:
example_index = 1
example_text = df_test.iloc[example_index]['article']
example_reference = df_test.iloc[example_index]['highlights']

In [195]:
print(example_text)

A drunk teenage boy had to be rescued by security after jumping into a lions' enclosure at a zoo in western India. Rahul Kumar, 17, clambered over the enclosure fence at the Kamla Nehru Zoological Park in Ahmedabad, and began running towards the animals, shouting he would 'kill them'. Mr Kumar explained afterwards that he was drunk and 'thought I'd stand a good chance' against the predators. Next level drunk: Intoxicated Rahul Kumar, 17, climbed into the lions' enclosure at a zoo in Ahmedabad and began running towards the animals shouting 'Today I kill a lion!' Mr Kumar had been sitting near the enclosure when he suddenly made a dash for the lions, surprising zoo security. The intoxicated teenager ran towards the lions, shouting: 'Today I kill a lion or a lion kills me!' A zoo spokesman said: 'Guards had earlier spotted him close to the enclosure but had no idea he was planing to enter it. 'Fortunately, there are eight moats to cross before getting to where the lions usually are and he

In [196]:
print(textwrap.fill(text=example_text, replace_whitespace=False,
                    fix_sentence_endings=True))

A drunk teenage boy had to be rescued by security after jumping into a
lions' enclosure at a zoo in western India.  Rahul Kumar, 17,
clambered over the enclosure fence at the Kamla Nehru Zoological Park
in Ahmedabad, and began running towards the animals, shouting he would
'kill them'. Mr Kumar explained afterwards that he was drunk and
'thought I'd stand a good chance' against the predators.  Next level
drunk: Intoxicated Rahul Kumar, 17, climbed into the lions' enclosure
at a zoo in Ahmedabad and began running towards the animals shouting
'Today I kill a lion!'  Mr Kumar had been sitting near the enclosure
when he suddenly made a dash for the lions, surprising zoo security.
The intoxicated teenager ran towards the lions, shouting: 'Today I
kill a lion or a lion kills me!'  A zoo spokesman said: 'Guards had
earlier spotted him close to the enclosure but had no idea he was
planing to enter it.  'Fortunately, there are eight moats to cross
before getting to where the lions usually are a

In [197]:
print(example_reference)

Drunk teenage boy climbed into lion enclosure at zoo in west India. Rahul Kumar, 17, ran towards animals shouting 'Today I kill a lion!' Fortunately he fell into a moat before reaching lions and was rescued.


In [198]:
print(textwrap.fill(text=example_reference, replace_whitespace=False,
                    fix_sentence_endings=True))

Drunk teenage boy climbed into lion enclosure at zoo in west India.
Rahul Kumar, 17, ran towards animals shouting 'Today I kill a lion!'
Fortunately he fell into a moat before reaching lions and was rescued.


In [199]:
def get_candidate_summary(results_path: str, record_index: int):
    results = pd.read_csv(results_path)
    candidate_summary = results.iloc[record_index]['pred_summary']
    rouge1 = results.iloc[record_index]['rogue1_f1']
    rouge2 = results.iloc[record_index]['rogue2_f1']
    rougeL = results.iloc[record_index]['rogueL_f1']
    print('ROUGE-1    ROUGE-2    ROUGE-L')
    print(f'{rouge1:.2f}       {rouge2:.2f}       {rougeL:.2f}')
    return candidate_summary

### Extractive Summerization

#### Method 1. TF-IDF summerization

In [200]:
method1_results_path = './results/method1-baseline/TFIDFsum.csv'
method1_summary = get_candidate_summary(results_path=method1_results_path,
                                        record_index=example_index)
method1_summary

ROUGE-1    ROUGE-2    ROUGE-L
0.39       0.15       0.35


"A police spokesman said: 'He has been cautioned and will be sent for psychiatric evaluation. The intoxicated teenager ran towards the lions, shouting: 'Today I kill a lion or a lion kills me!' Mr Kumar explained afterwards that he was drunk and 'thought I'd stand a good chance' against the predators. A drunk teenage boy had to be rescued by security after jumping into a lions' enclosure at a zoo in western India. "

In [201]:
print(textwrap.fill(text=method1_summary, replace_whitespace=False,
                    fix_sentence_endings=True))

A police spokesman said: 'He has been cautioned and will be sent for
psychiatric evaluation.  The intoxicated teenager ran towards the
lions, shouting: 'Today I kill a lion or a lion kills me!'  Mr Kumar
explained afterwards that he was drunk and 'thought I'd stand a good
chance' against the predators.  A drunk teenage boy had to be rescued
by security after jumping into a lions' enclosure at a zoo in western
India.


#### Method 2. BERTsum

In [202]:
method2_results_path = './results/method2-BERTsum/BERTsum.csv'
method2_summary = get_candidate_summary(results_path=method2_results_path,
                                        record_index=example_index)
method2_summary

ROUGE-1    ROUGE-2    ROUGE-L
0.41       0.11       0.34


"Rahul Kumar , 17 , clambered over the enclosure fence at the \xa0 Kamla Nehru Zoological Park in Ahmedabad , and began running towards the animals , shouting he would ' kill them '. Mr Kumar explained afterwards that he was drunk and ' thought I 'd stand a good chance ' against the predators. Next level drunk : Intoxicated Rahul Kumar , 17 , climbed into the lions ' enclosure at a zoo in Ahmedabad and began running towards the animals shouting ' Today I kill a lion ! '."

In [203]:
print(textwrap.fill(text=method2_summary, replace_whitespace=False,
                    fix_sentence_endings=True))

Rahul Kumar , 17 , clambered over the enclosure fence at the   Kamla
Nehru Zoological Park in Ahmedabad , and began running towards the
animals , shouting he would ' kill them '. Mr Kumar explained
afterwards that he was drunk and ' thought I 'd stand a good chance '
against the predators.  Next level drunk : Intoxicated Rahul Kumar ,
17 , climbed into the lions ' enclosure at a zoo in Ahmedabad and
began running towards the animals shouting ' Today I kill a lion ! '.


#### Method 3-1. BART-CNN

In [204]:
bart_results_path = './results/method3-bart-large-cnn/bart-large-cnn_summary.csv'
bart_summary = get_candidate_summary(results_path=bart_results_path,
                                     record_index=example_index)
bart_summary

ROUGE-1    ROUGE-2    ROUGE-L
0.49       0.27       0.47


"Rahul Kumar, 17, clambered over enclosure fence at\xa0Kamla Nehru Zoological Park. He ran towards the animals, shouting: 'Today I kill a lion or a lion kills me!' Fortunately, Mr Kumar fell into a moat before reaching the animals. He has been cautioned and sent for psychiatric evaluation."

In [205]:
print(textwrap.fill(text=bart_summary, replace_whitespace=False,
                    fix_sentence_endings=True))

Rahul Kumar, 17, clambered over enclosure fence at Kamla Nehru
Zoological Park.  He ran towards the animals, shouting: 'Today I kill
a lion or a lion kills me!'  Fortunately, Mr Kumar fell into a moat
before reaching the animals.  He has been cautioned and sent for
psychiatric evaluation.


#### Method 3-2. T5-small

In [206]:
t5_results_path = './results/method3-t5-small/t5-small_summary.csv'
t5_summary = get_candidate_summary(results_path=t5_results_path,
                                        record_index=example_index)
t5_summary

ROUGE-1    ROUGE-2    ROUGE-L
0.51       0.24       0.49


"Rahul Kumar, 17, climbed into enclosure fence at zoo in western india. he ran towards the animals shouting: 'Today I kill a lion or a lion kills me!' zoo guards acted quickly enough to prevent a tragedy similar to that in Delhi."

In [207]:
print(textwrap.fill(text=t5_summary, replace_whitespace=False,
                    fix_sentence_endings=True))

Rahul Kumar, 17, climbed into enclosure fence at zoo in western india.
he ran towards the animals shouting: 'Today I kill a lion or a lion
kills me!'  zoo guards acted quickly enough to prevent a tragedy
similar to that in Delhi.
